#### Setup

In [1]:
import tensorflow as tf
import numpy as np
import os
import time

#### Download Shakespeare dataset

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1130496/1115394 [==============================] - 0s 0us/step


In [3]:
text = open(path_to_file, "rb").read().decode(encoding="utf-8")
print(f"Length of the text: {len(text)}")

Length of the text: 1115394


In [4]:
# Check first 250 chars
text[:250]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n'

In [5]:
# Check unique chars in the file
vocab = sorted(set(text))
print(f"Length of unique chars: {len(vocab)}")
print(f"First 5 chars: {vocab[:5]}")

Length of unique chars: 65
First 5 chars: ['\n', ' ', '!', '$', '&']


#### Process text

In [6]:
example_texts = ["abcdefg", "xyz"]

chars = tf.strings.unicode_split(example_texts, input_encoding="UTF-8")
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [7]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)

In [8]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [9]:
chars_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [10]:
# You can tf.strings.reduce_join to join the characters back into strings.
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [11]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

- ids_from_chars layer
- chars_from_ids layer
- text_from_ids function

#### Create training examples and targets

In [12]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, "UTF-8"))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

for ids in ids_dataset.take(10):
    print(f"{chars_from_ids(ids).numpy().decode('utf-8')}")

F
i
r
s
t
 
C
i
t
i


In [13]:
seq_length = 100
example_per_epoch = len(text) // (seq_length+1)

sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)
for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [14]:
for seq in sequences.take(5):
    print(f"{text_from_ids(seq).numpy()}")

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [15]:
def split_input_target(sequence):
    return sequence[:-1], sequence[1:]

In [16]:
print(split_input_target(list("Tensorflow")))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'], ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])


In [17]:
dataset = sequences.map(split_input_target)

In [18]:
for input_example, target_example in dataset.take(1):
    print(f"Input: {text_from_ids(input_example).numpy()}")
    print(f"Targtet: {text_from_ids(target_example).numpy()}")

Input: b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Targtet: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


#### Create training batches

In [19]:
BATCH_SIZE = 64
BUFFER_SIZE = 1000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

#### Build the model

In [20]:
vocab_size = len(vocab)
embedding_size = 256
rnn_units = 1024


class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.gru = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [21]:
model = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_size=embedding_size,
    rnn_units=rnn_units
)

In [22]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [23]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


In [24]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices 

array([21, 53, 19, 43, 38,  4, 13, 55, 10, 28, 25, 54, 49, 41, 18,  3, 45,
       22, 22, 10,  7, 22,  1, 26, 51, 18, 65, 60, 51, 51, 13, 22,  4, 35,
       28, 41, 53,  6, 31, 19, 29, 63, 56, 40,  6, 40, 57,  7, 12, 47, 29,
        2, 33, 25, 52, 60, 45, 31, 58, 48, 36, 64,  8, 36,  5,  9, 61, 32,
       23, 13, 28, 43, 48, 44, 48, 14, 59, 11, 31, 11, 32, 10, 26, 45,  9,
       17, 21,  6, 60, 24,  5, 10,  3, 43, 20, 19, 41, 63,  4, 42])

In [25]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'nd so are like to do.\n\nCOMINIUS:\nThat is the way to lay the city flat;\nTo bring the roof to the foun'

Next Char Predictions:
 b"HnFdY$?p3OLojbE!fII3,I\nMlEzull?I$VObn'RFPxqa'ar,;hP TLmufRsiWy-W&.vSJ?OdieiAt:R:S3Mf.DH'uK&3!dGFbx$c"


#### Train the model

In [26]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

for x, y in dataset.take(1):
    y_hat = model(y)    
    mean_loss = loss(y, y_hat).numpy().mean()

    print(f"Mean loss: {mean_loss}")

Mean loss: 4.191269397735596


In [27]:
model.compile(optimizer="adam", loss=loss)

In [28]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [29]:
EPOCHS = 20
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 28s 137ms/step - loss: 2.7109
Epoch 2/20
172/172 [==============================] - 24s 136ms/step - loss: 1.9940
Epoch 3/20
172/172 [==============================] - 23s 130ms/step - loss: 1.7297
Epoch 4/20
172/172 [==============================] - 24s 136ms/step - loss: 1.5627
Epoch 5/20
172/172 [==============================] - 23s 134ms/step - loss: 1.4570
Epoch 6/20
172/172 [==============================] - 23s 130ms/step - loss: 1.3860
Epoch 7/20
172/172 [==============================] - 22s 128ms/step - loss: 1.3295
Epoch 8/20
172/172 [==============================] - 22s 129ms/step - loss: 1.2838
Epoch 9/20
172/172 [==============================] - 22s 129ms/step - loss: 1.2403
Epoch 10/20
172/172 [==============================] - 23s 130ms/step - loss: 1.1994
Epoch 11/20
172/172 [==============================] - 23s 130ms/step - loss: 1.1587
Epoch 12/20
172/172 [==============================] - 23s 130ms/step - lo

#### Generate Text

In [31]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [32]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [33]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
Thou hadst a hundred pound of a man: would I were,
Sugarsong them! indeed, indeed, rascals,
And fact her with thy love, thy father welcome.

Second Keeper:
Ay, then; he sits in it.

CURTIS:
Why, ever as the moon that did he till the office
or that tide. You pluck my master gown.

KATHARINA:
Bethinks she's madle. Thou dogs, to't, strike thee well, for I am doubt.

GREMIO:
Take heed, Somerset good man; pray?
Then to go hurl of such gentlewoman,
So do not rich their loss imagination crows.

ABHORSON:
How fares your son?

TRANIO:
And yet I think not rise and brand to reath a
begging.

ANTO:
It is my love to thee and fell to mine,
That makes them oft whereof we are feasts must:
And, boy, and must Katharina shall without
Be brief that wedemeth brain'd upon his side,
to instruments shallongs. They come to me,
My son and permit: a reason so
Then, As bell after to the court!
Come, I'll ame death; for the life
Of Sicilia,--
Better but too condline with a passing eat;
Earines to you both. 

#### Reference:

1. [Text generation with an RNN](https://www.tensorflow.org/text/tutorials/text_generation)